In [3]:
import mitreattack
from stix2 import MemoryStore
from stix2 import Filter
from stix2 import FileSystemSource


In [4]:
# 通过bundle加载数据
src = MemoryStore()
src.load_from_file("../assets/attck_database/attack-stix-data/enterprise-attack/enterprise-attack.json")

In [5]:
# 移除弃用和撤销的对象
from stix2 import Filter

def remove_revoked_deprecated(stix_objects):
    """Remove any revoked or deprecated objects from queries made to the data source"""
    # Note we use .get() because the property may not be present in the JSON data. The default is False
    # if the property is not set.
    return list(
        filter(
            lambda x: x.get("x_mitre_deprecated", False) is False and x.get("revoked", False) is False,
            stix_objects
        )
    )

#datas = src.query([ Filter("type", "=", "course-of-action") ])
#datas = remove_revoked_deprecated(datas)

In [8]:
# 获取所有Tactic
from stix2 import Filter

def getTacticsByMatrix(thesrc):
    tactics = {}
    matrix = thesrc.query([
        Filter('type', '=', 'x-mitre-matrix'),
    ])

    # 丢弃启用数据
    matrix= remove_revoked_deprecated(matrix)

    for i in range(len(matrix)):
        tactics[matrix[i]['name']] = []
        for tactic_id in matrix[i]['tactic_refs']:
            tactics[matrix[i]['name']].append(thesrc.get(tactic_id))

    return tactics

# get tactic layout
data_dict = getTacticsByMatrix(src)




# 提取所有的 "name" 字段和对应的 "external_id" 值
name_external_id_pairs = []
for item in data_dict['Enterprise ATT&CK']:
    name = item['name']
    stix_id = item['id']
    external_id = item['external_references'][0]['external_id']
    name_external_id_pairs.append((name, external_id,stix_id))

# 打印结果
for name, external_id,stix_id in name_external_id_pairs:
    print(f"# {name}")
    print(f"> External ID: {external_id}")
    print(f"> STIX ID: {stix_id}")

# Reconnaissance
> External ID: TA0043
> STIX ID: x-mitre-tactic--daa4cbb1-b4f4-4723-a824-7f1efd6e0592
# Resource Development
> External ID: TA0042
> STIX ID: x-mitre-tactic--d679bca2-e57d-4935-8650-8031c87a4400
# Initial Access
> External ID: TA0001
> STIX ID: x-mitre-tactic--ffd5bcee-6e16-4dd2-8eca-7b3beedf33ca
# Execution
> External ID: TA0002
> STIX ID: x-mitre-tactic--4ca45d45-df4d-4613-8980-bac22d278fa5
# Persistence
> External ID: TA0003
> STIX ID: x-mitre-tactic--5bc1d813-693e-4823-9961-abf9af4b0e92
# Privilege Escalation
> External ID: TA0004
> STIX ID: x-mitre-tactic--5e29b093-294e-49e9-a803-dab3d73b77dd
# Defense Evasion
> External ID: TA0005
> STIX ID: x-mitre-tactic--78b23412-0651-46d7-a540-170a1ce8bd5a
# Credential Access
> External ID: TA0006
> STIX ID: x-mitre-tactic--2558fd61-8c75-4730-94c4-11926db2a263
# Discovery
> External ID: TA0007
> STIX ID: x-mitre-tactic--c17c5845-175e-4421-9713-829d0573dbc9
# Lateral Movement
> External ID: TA0008
> STIX ID: x-mitre-tactic--71